use Soulsby (Dynamics of Marine Sands) text section 5.3 to compute bed shear stress (mean and max) from waves and currents

below is an implementation of values in example 5.1 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%run -i sedfuncs.py

start out by using the values in the Soulsby text: example 5.1

In [2]:
uw = 0.5 # m s-1, near-bed orbital velocity
T = 12.6 # s, wave period
phi = 30 # deg, relative angle to currents
U = 1 # m s-1, depth averaged current magnitude
z0 = 0.001 # m, roughness length of hydrodynamically rough bed
h = 10 # m, water depth
rho = 1027 # kg m-3, water density

In [3]:
# calculate Cd using z0/h; table 10
z0/h

0.0001

In [4]:
Cd = 0.00237 # Grant-Madsen or DATA 2

In [5]:
# calculate wave friction factor fw; table 10
A = uw*T/(2*np.pi)
A/z0

1002.6761414789406

In [6]:
fw = 0.0316 # Grant-Madsen


In [7]:
# current only bed shear stress
tauc = rho*Cd*U**2 # N m-2
tauc

2.43399

In [8]:
# wave-only peak bed shear stress
tauw = 0.5*rho*fw*uw**2 # N m-2
tauw

4.05665

In [9]:
# proportion of wave and current stresses: X
X = tauc/(tauc+tauw)
X

0.37499999999999994

In [10]:
# tau max cooefficient values for the chosen model: Grant-Madsen
a1 = 0.11
a2 = 1.95
a3 = -0.49
a4 = -0.28
m1 = 0.65
m2 = -0.22
m3 = 0.15
m4 = 0.06
n1 = 0.71
n2 = -0.19
n3 = 0.17
n4 = -0.15
I = 0.67

In [11]:
# tau mean coefficient values for the chosen model: Grant-Madsen
b1 = 0.73
b2 = 0.40
b3 = -0.23
b4 = -0.24
p1 =  -0.68
p2 = 0.13
p3 = 0.24
p4 = -0.07
q1 = 1.04
q2 = -0.56
q3 = 0.34
q4 = -0.27
J = 0.50

In [12]:
np.cos(phi*(2*np.pi)/360)

0.8660254037844387

In [13]:
# compute tau_max and tau_mean coeffs

In [14]:
def tau_coeffs(c1,c2,c3,c4,K,phi,fw,Cd):
    """
    compute c = (c1 + c2|cos phi|^K) + (c3 + c4|cos phi|^K)*log10(fw/Cd)
    Soulsby, Dynamics of Marine Sands, pg 94

    phi is in degrees

    needed for parameters a, m, n, and I for Z computation
    or for parameters b, p, q, and J for Y computation
    """
    phi_rad = phi*(2*np.pi)/360
    
    c = (c1+c2*np.cos(phi_rad)**K)+((c3+c4*np.cos(phi_rad)**K)*np.log10(fw/Cd))
    return c

In [15]:
a = tau_coeffs(a1,a2,a3,a4,I,phi,fw,Cd)
a

1.0435796011844096

In [16]:
m = tau_coeffs(m1,m2,m3,m4,I,phi,fw,Cd)
m

0.6802484444717939

In [17]:
n = tau_coeffs(n1,n2,n3,n4,I,phi,fw,Cd)
n

0.5754581680155827

In [18]:
b = tau_coeffs(b1,b2,b3,b4,J,phi,fw,Cd)
b

0.5922564051457643

In [19]:
p = tau_coeffs(p1,p2,p3,p4,J,phi,fw,Cd)
p

-0.36231721334432543

In [20]:
q = tau_coeffs(q1,q2,q3,q4,J,phi,fw,Cd)
q

0.6186846166400837

In [21]:
Z = 1+a*X**m*(1-X)**n
Z

1.4086001319792205

In [22]:
Y = X*(1+b*X**p*(1-X)**q)
Y

0.6119141197266875

In [23]:
taumax = Z*(tauc+tauw) # N m-2
taumax

9.14271636062961

The value in the text is taumax = 9.15 N m-2

In [24]:
taumean = Y*(tauc+tauw) # N m-2
taumean

3.9717142620628274

The value in the text is taumean = 3.97 N m-2

Try again for the DATA2 method

In [25]:
fw = 0.0383 # DATA 2
# Cd is the same for Grant-Madsen
# therefore, tauc is also the same
tauw = 0.5*rho*fw*uw**2 # N m-2
tauw

4.9167625

In [26]:
# DATA2 model

In [27]:
taumean = tauc*(1+1.2*(tauw/(tauc+tauw))**3.2)
taumean

3.2405038180099037

The value in the text is taumean = 3.24 N m-2

In [28]:
taumax = ((taumean + tauw*np.cos(phi*(2*np.pi)/360))**2 + (tauw*np.sin(phi*(2*np.pi)/360))**2)**0.5
taumax

7.89124934329207

The value in the text is taumax = 7.89 N m-2